In [1]:
import sys
import torch
import matplotlib.pyplot as plt
from matplotlib import patches
import pandas as pd
import numpy as np
import pandas as pd
from tqdm import *
import cv2
import numpy 

def get_rectangle_edges_from_pascal_bbox(bbox):
    xmin_top_left, ymin_top_left, xmax_bottom_right, ymax_bottom_right = bbox

    bottom_left = (xmin_top_left, ymax_bottom_right)
    width = xmax_bottom_right - xmin_top_left
    height = ymin_top_left - ymax_bottom_right

    return bottom_left, width, height

def draw_pascal_voc_bboxes(
    plot_ax,
    bboxes,
    labels=None,
    get_rectangle_corners_fn=get_rectangle_edges_from_pascal_bbox,
):
    for idx,bbox in enumerate(bboxes):
        bottom_left, width, height = get_rectangle_corners_fn(bbox)

        rect_1 = patches.Rectangle(
            bottom_left,
            width,
            height,
            linewidth=4,
            edgecolor="black",
            fill=False,
        )
        rect_2 = patches.Rectangle(
            bottom_left,
            width,
            height,
            linewidth=2,
            edgecolor="white",
            fill=False,
        )

        # Add the patch to the Axes
        plot_ax.add_patch(rect_1)
        plot_ax.add_patch(rect_2)
        if labels is not None:
            plot_ax.annotate(labels[idx], (bbox[0], bbox[1]-10), color='w', weight='bold', 
                    fontsize=12, ha='center', va='center')

def show_image(
    image, bboxes=None,labels=None, draw_bboxes_fn=draw_pascal_voc_bboxes, figsize=(10, 10)
):
    fig, ax = plt.subplots(1, figsize=figsize)
    ax.imshow(image)

    if bboxes is not None:
        draw_bboxes_fn(ax, bboxes ,labels)

    plt.show()


def load_model(ckpt_path, conf=0.01, iou=0.5):
    model = torch.hub.load('yolov7/',
                           'custom',
                           path_or_model=ckpt_path,
                           source='local',
                           force_reload=True)  # local repo
    model.conf = conf  # NMS confidence threshold
    model.iou  = iou  # NMS IoU threshold
    model.classes = None   # (optional list) filter by class, i.e. = [0, 15, 16] for persons, cats and dogs
    model.multi_label = False  # NMS multiple labels per box
    model.max_det = 1000  # maximum number of detections per image
    return model


In [29]:
FOLD=5

In [30]:
model = load_model(f'runs/train/yolov7ve6e_fold_{FOLD}/weights/best.pt')


                 from  n    params  module                                  arguments                     
  0                -1  1         0  models.common.ReOrg                     []                            
  1                -1  1      8800  models.common.Conv                      [12, 80, 3, 1]                
  2                -1  1     70880  models.common.DownC                     [80, 160, 1]                  
  3                -1  1     10368  models.common.Conv                      [160, 64, 1, 1]               
  4                -2  1     10368  models.common.Conv                      [160, 64, 1, 1]               
  5                -1  1     36992  models.common.Conv                      [64, 64, 3, 1]                
  6                -1  1     36992  models.common.Conv                      [64, 64, 3, 1]                
  7                -1  1     36992  models.common.Conv                      [64, 64, 3, 1]                
  8                -1  1     36992  

Adding autoShape... 


In [31]:
# preds

In [32]:
# %matplotlib inline
# import cv2
# import numpy
# image=cv2.imread('test/images/1000.jpg')
# pix = numpy.array(image)
# img = pix[...,::-1]

# height, width = img.shape[:2]
# results = model(img, size=1280, augment=False)  # custom inference size
# print(results)

# preds   = results.pandas().xyxy[0]
# bboxes  = preds[['xmin','ymin','xmax','ymax']].values
# labels=preds.name.values
# show_image(img,bboxes,labels)

In [33]:
test = pd.read_csv('sample_submission/sample_submission.csv')

predicted_frame=pd.DataFrame()
for i in tqdm_notebook(test.ImageID.unique()):
    image=cv2.imread('test/images/'+i)
    pix = numpy.array(image)
    img = pix[...,::-1]
    height, width = img.shape[:2]
    results = model(img, size=1280, augment=True)  # custom inference size
    preds   = results.pandas().xyxy[0]
    preds.columns=['XMin', 'YMin', 'XMax', 'YMax', 'Conf','label' ,'LabelName']
    preds['ImageID'] = i
    predicted_frame = pd.concat([predicted_frame , preds])

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  after removing the cwd from sys.path.


  0%|          | 0/555 [00:00<?, ?it/s]

In [34]:
predicted_frame.shape

(3554, 8)

In [36]:
predicted_frame[test.columns].to_csv(f'final_subs/subs_fold_{FOLD}.csv',index=None)

In [35]:
!gsutil cp runs/train/yolov7ve6e_fold_5/weights/best.pt gs://community-ds/inference_models/sordi/av/best_5.pt

Copying file://runs/train/yolov7ve6e_fold_4/weights/best.pt [Content-Type=application/octet-stream]...
==> NOTE: You are uploading one or more large file(s), which would run          
significantly faster if you enable parallel composite uploads. This
feature can be enabled by editing the
"parallel_composite_upload_threshold" value in your .boto
configuration file. However, note that if you do this large files will
be uploaded as `composite objects
<https://cloud.google.com/storage/docs/composite-objects>`_,which
means that any user who downloads such objects will need to have a
compiled crcmod installed (see "gsutil help crcmod"). This is because
without a compiled crcmod, computing checksums on composite objects is
so slow that gsutil disables downloads of composite objects.

\ [1 files][315.7 MiB/315.7 MiB]                                                
Operation completed over 1 objects/315.7 MiB.                                    
